## Employee Reviews da Microsoft 
### Objetivo:
Desenvolver um sistema que analisa avaliações de funcionários da Microsoft, processa os textos e classifica automaticamente cada review como positivo ou negativo, com base no conteúdo textual.

### Coleta e Organização dos Dados

In [4]:
import pandas as pd

df = pd.read_csv("employee_reviews.csv")

# Filtrando as avaliações da Microsoft
df = df[df['company'].str.lower() == 'microsoft'].reset_index(drop=True)

df.columns


Index(['Unnamed: 0', 'company', 'location', 'dates', 'job-title', 'summary',
       'pros', 'cons', 'advice-to-mgmt', 'overall-ratings',
       'work-balance-stars', 'culture-values-stars',
       'carrer-opportunities-stars', 'comp-benefit-stars',
       'senior-mangemnet-stars', 'helpful-count', 'link'],
      dtype='object')

### Pré-processamento de Texto

In [6]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Preparar
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Juntar os textos
df['text'] = df[['summary', 'pros', 'cons']].fillna('').agg(' '.join, axis=1)

# Função de limpeza
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # remove pontuação
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return ' '.join(tokens)

# Aplicar
df['clean_text'] = df['text'].apply(preprocess)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rebek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rebek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rebek\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rebek\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\rebek/nltk_data'
    - 'c:\\Users\\rebek\\OneDrive\\Área de Trabalho\\Reviews_Microsoft\\venv\\nltk_data'
    - 'c:\\Users\\rebek\\OneDrive\\Área de Trabalho\\Reviews_Microsoft\\venv\\share\\nltk_data'
    - 'c:\\Users\\rebek\\OneDrive\\Área de Trabalho\\Reviews_Microsoft\\venv\\lib\\nltk_data'
    - 'C:\\Users\\rebek\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


### Criação dos Rótulos (Positivo/Negativo)

In [ ]:
# Rótulo binário: 1 = positivo (rating >= 4), 0 = negativo (rating <= 2)
df = df[df['overall-ratings'].isin([1.0, 2.0, 4.0, 5.0])]
df['label'] = df['overall-ratings'].apply(lambda x: 1 if x >= 4 else 0)


### Extração de Features

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['clean_text'])
y = df['label']


### Modelagem e Classificação

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


### Simular Novos Reviews

In [ ]:
def predict_review(text):
    clean = preprocess(text)
    vec = vectorizer.transform([clean])
    prediction = model.predict(vec)
    return "Positivo" if prediction[0] == 1 else "Negativo"

# Exemplo
predict_review("The team is very collaborative, and the work-life balance is great.")


### Visualização e Interface

In [ ]:
import gradio as gr
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

# Certifique-se de ter feito isso antes (no seu outro arquivo):
# joblib.dump(model, "modelo_sentimento.pkl")
# joblib.dump(tfidf, "vetorizador_tfidf.pkl")

# Carregando o modelo e o vetor TF-IDF
model = joblib.load("modelo_sentimento.pkl")
tfidf = joblib.load("vetorizador_tfidf.pkl")

# Baixar recursos necessários
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Função de pré-processamento
def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in string.punctuation]
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return ' '.join(tokens)

# Função para a interface
def classificar_review(review_text):
    texto_limpo = preprocess(review_text)
    vetor = tfidf.transform([texto_limpo])
    pred = model.predict(vetor)[0]
    prob = model.predict_proba(vetor).max()
    return f"Sentimento: {'Positivo' if pred == 1 else 'Negativo'} (confiança: {prob:.2f})"

# Interface com Gradio
demo = gr.Interface(
    fn=classificar_review,
    inputs=gr.Textbox(lines=5, label="Digite um review da Microsoft"),
    outputs=gr.Textbox(label="Resultado da Classificação"),
    title="Classificador de Reviews - Microsoft",
    description="Este modelo classifica automaticamente se um review é positivo ou negativo com base em PLN."
)

# Executa o app
if __name__ == "__main__":
    demo.launch()
